# Module 5: State tracking and Control

This chapter contains information that you will need when preparing for the second part: the final challenge. The ultimate goal is to let KITT drive from an initial (known) location A to a given target location B. In this final challenge you are on your own and no explicit steps are provided to help you through the process. Nevertheless, a few hints are given in this chapter on how you could attack the problem (but many alternative solution approaches exist).

**Learning objectives:** Basics of system engineering. Extension of control theory knowledge. 

**Deliverables:** 
- Sections of your final report regarding the problem analysis and overall design, and the
more detailed design of the control system.
- Python code for control that was tested on your virtual car model.

**Preparation:** Read the chapter. This module provides ideas you may use.

**What is needed:** Python code that implements your virtual car model. 

**Time duration:** One session for brainstorming (high-level system design). Two sessions for developing
the control system in Python and debugging it. Two or more sessions (evolving into system integration) for trajectory control analysis, development, and verification. Three homework sessions for study and report writing.

## Defining your approach

At the beginning of this second part, it merits to brainstorm with your entire group on the approach
that you will take. For your selected approach, determine whether it will do the job (and under what assumptions/conditions). You can opt for a simple and robust “combinatorial” approach (i.e., based on
many if-then-else statements), that will be able to handle the expected situations but not more than that,
or a more thoughtful approach based on control theory, that is much more robust but also more risky to
implement. 

This high-level system design is part of system engineering, i.e., with the entire system in mind, define
an approach at a high level that will do the job, and define specifications of each constituting subsystem.
These parts can then be designed by specialists that don’t have to know or consider the entire system. If
the parts are tested and verified to meet their specifications, then the entire system is supposed to work.
(Or so you would think.)

**Deliverable**

In your final report, document your selected approach, and the alternatives that you considered, plus your
motivation for the selection.

Document the foreseen consequences of this choice. Draw a block scheme that shows what software
blocks should interact, and define the interaction (e.g., variables).

Your solution will probably contain a finite state machine and/or a loop. An important consideration is
the timing of this loop. Document your analysis of this in sufficient detail. How often do you intend to
measure your location, keeping in mind the constraints and trade-offs here. Consider also the various
delays in the system. If you obtain a location fix, how old is that information? If you estimate velocity
from two locations, then to what time point does that velocity refer? How fast should one iteration of the
loop be? (If it is too fast, then you won’t have new location information, but if it is too slow, then you
might miss your target.) Can you merge location fixes from the audio beacon with predictions of your
position using your car model?

The text on this in your final report can be placed into an initial section “Problem definition and analysis”, or “Problem analysis and high-level design”, depending on how you want to organize your report. Alternatively, it can be placed after the localization and car modeling sections, if you need to use information from those sections.


## Defining a target path to the goal

Although not necessary, it is helpful to create a target trajectory from A to B. Assume that you know your
starting position and orientation (i.e., the initial state). From here, you can draw a feasible trajectory to the target *B* in many ways.

- You have some freedom in your orientation when you arrive at B.
- Since steering amounts to driving on circles, you can define a circle with the smallest possible
radius and then drive straight, or define a circle with a larger radius that goes through both given
points and is tangential to the given orientation, or anything in between.
- In some cases you may need to drive backwards to be able to reach the goal. E.g., what happens
if the target is within the smallest turning circle? [Note: This scenario is not part of the basic
challenges, but you could use it for the Free Challenge. See Chapter 9.(@B update)]
- Depending on your control strategy, you may need to recreate the target trajectory each time new
location information is measured.

**Deliverable** 

In the final report, document your path planning solution. Illustrate this with examples of generated routes under varying conditions. 

## Getting to your goal

Assume that we are able to keep KITT on any trajectory. Obviously, we have to slow down when
approaching our target position. In one dimension and with a known position, velocity and braking
curve, it is quickly clear how we can find the moment to brake. In two-dimensional space, this is much
harder: this is called a *non-holonomic system*.

### Partitioning by projection

A solution to this problem is given by projection. If we could project our two-dimensional space onto
a one-dimensional space S, then we could use KITT’s position in S for one-dimensional control. Figure
8.1 (@B opdate) depicts this idea. The line represents the trajectory that we would want to follow.

Let KITT's trajectory be given by, 

$$ \mathbf{x}(t) = \left[ x(t), y(t) \right]^T $$

We define the projection of \((x, y)\) onto \(S\) by
$$
z = z(x, y)
$$

where $z \in S$. The projected position $z$ can be regarded as the distance we travel on $ S $. KITT's speed should then be continuously controlled so that $z$ approaches the desired distance without overshooting. Thus, using $z$ as a measure for distance allows for one-dimensional control in a plane.

This approach requires knowing KITT's trajectory: we need an *estimate* of KITT's future movement. This is given by the planned trajectory. We can now state our wanted projection: The projected position $z$ is given by the arc length of the planned trajectory from KITT's current position to its target position. This allows for velocity control along any trajectory in a plane.

```{figure} trajectory.png
---
height/width: 150px
name: KITTtraject-figure
---
KITT's trajectory
```

### Following your goal

In the previous section, we assumed that we were able to let KITT follow any predefined trajectory. We will now design a controller which keeps KITT on track.

Intuitively, you will think of a solution where you know your current position/orientation, and you always steer towards the target. Once you are oriented towards the target, your "angle error" is zero, and you just have to drive straight. In all other cases, the angle error determines how much you need to steer. It is easy to see that this approach might work, but also might become unstable once you are very close to the target.

Suppose KITT is driving on its trajectory $S$, where its orientation with respect to the $x$-axis is given by $\theta$. Let the angle tangent to its trajectory $S$ be given by $\theta_t$. Ideally, this angle should be equal to KITT's orientation. If KITT's orientation deviates from this angle, KITT should turn its wheels to steer back. This motivates a feedback law given by

$$
\phi = -k(\theta - \theta_t)
$$

for a positive $k$. Substituting in Equation (\ref{eq:steering-derivative}) yields the autonomous non-linear system

$$
L \,\frac{d}{dt}\theta \;+\; v \sin(k(\theta-\theta_t)) = 0\,.
$$

We should choose $k$ such that this system is asymptotically stable. To investigate the stability, we introduce a potential function (error function) given by

$$
T(\theta) = \frac{1}{2}(\theta-\theta_t)^2 \,.
$$

A first observation is that $T(\theta)=0$ if and only if $\theta=\theta_t$, which is exactly what we want. Second, notice that $T(\theta) \ge 0$. We can conclude that $T$'s minimum corresponds to our equilibrium point. Notice that

$$
\frac{d}{dt} T(\theta) = -(\theta-\theta_t)\frac{v \sin{(k(\theta-\theta_t))}}{L}.
$$

Figure \ref{fig:KITTpotential-figure} depicts both \(T(\theta)\) and its time-derivative.

```{figure} potentialfunction.png
---
height/width: 150px
name: KITTpotential-figure
---
Graph of potential function and it's derivative
```

Consider KITT's orientation at any instant. If $ |\theta| < \theta_m $, then by Figure \ref{fig:lyaponuv} the potential function will have a negative derivative. But then it will decrease over time and will keep decreasing until $\theta = \theta_t$. So in conclusion, if $ |\theta| < \theta_m $, then $\theta$ will converge to its equilibrium point. By inspection of Equation (\ref{eq:lyapunov}) we can now state that Equation (\ref{eq:angle-diffeq}) is locally asymptotically stable for any

$$
-\frac{\pi}{k} < \theta - \theta_t < \frac{\pi}{k}.
$$

This treatment is also called the investigation of *Lyapunov stability*, where $T$ is called the *Lyapunov function*. It is an extensive topic in the control literature.

### Deliverable

Implement your control algorithm and test it using your virtual car model. (@ provide some stuff to guid them how) In your final report, document the results of the tests, which prove that given correct position estimation, the control algorithm will get the car to its goal.

## Optional: State tracking and state error feedback

In the signal processing/control literature, Kalman filters are used to estimate the state (position, velocity) of a system, given its inputs and observed (noisy) outputs. This assumes linear systems in state-space form (either continuous-time or discrete-time): in discrete time, we model

$$
\left\{
\begin{array}{lcl}
  \mathbf{x}_k &=& \mathbf{A} \mathbf{x}_{k-1} + \mathbf{B} \mathbf{u}_k + \mathbf{w}_k \\
  \mathbf{y}_k  &=& \mathbf{C} \mathbf{x}_k + \mathbf{v}_k
\end{array}
\right.
$$

Here, $\mathbf{u}_k$ is the system input at time $k$ (your velocity and steering commands), $\mathbf{x}$ is the (true) state vector, and $\mathbf{y}_k$ is an observation, in our case your position estimator. For example, for moving in 1D on a line, without friction, and with a discretization time step $\Delta T$,

$$
\mathbf{x} = \left[\begin{array}{c}
   z \\
   v
  \end{array}\right] \,, \qquad
\mathbf{A} = \left[\begin{array}{cc}
     1 & \Delta T \\
 0 & 1
\end{array}\right] \,, \qquad
\mathbf{u} = F_a \,, \qquad
\mathbf{B} = \left[\begin{array}{c}
    0 \\ \frac{1}{m} \end{array}\right] \,, \qquad
\mathbf{C} = [1 \quad 0] \,.
$$

The disturbances $\mathbf{v}_k$ and $\mathbf{w}_k$ are unknown additive noise processes at the input and the output, respectively, that will make the state and the output deviate from their real values. The Kalman filter is an estimator for $\mathbf{x}_k$ given $\mathbf{u}_k$ and $\mathbf{y}_k$ (and the previous state estimate at time $k-1$); it tries to track this state; it is a topic for MSc-level courses (e.g., EE4C03). The fundamental idea here is that we do not have access to the true state $\mathbf{x}_k$, we only have estimates $\hat{\mathbf{x}}_k$ that we keep updated.

If the observation noise $\mathbf{v}_k$ is high, then we don't trust the measurements $\mathbf{y}_k$, and will estimate the next state as $\hat{\mathbf{x}}_k = \mathbf{A} \hat{\mathbf{x}}_{k-1} + \mathbf{B} \mathbf{u}_k$, i.e., using the model predictions only. On the other hand, if the input (state) noise $\mathbf{w}_k$ is high, we don't trust the model state update and estimate the state (position, velocity) mostly from the observations. The Kalman filter gives a weighted average of these two extremes, using something called the Kalman gain. Our trust is modeled by covariance matrices $\mathbf{Q}_w = \mathbb{E}[\mathbf{w}_k\mathbf{w}_k^T]$ and $\mathbf{Q}_v = \mathbb{E}[\mathbf{v}_k\mathbf{v}_k^T]$. For example, $\mathbf{Q}_w$ contains on its main diagonal the measurement noise variance on your position estimates, which you documented in the preceding Modules. $\mathbf{Q}_v$ is the jitter on your input, and you could measure this by repeatedly giving the same control inputs and seeing how the state varies (measuring the state with a ruler rather than the less accurate beacon locator).

Kalman filters are described in many books, see e.g. Chapter 7.4 in

> Monson H. Hayes, *Statistical digital signal processing and modeling*, John Wiley and Sons, New York, 1996. ISBN: 0-471-59431-8.

Unfortunately, steering a car is highly nonlinear. You could try to use an Extended Kalman filter. Alternatively, since you have your own car model, you can use that to predict the next location and state, given the same inputs, and use the observed error in position to update/correct your state estimate, similar to what a Kalman filter does.

A complete theory is not developed here, the above is meant as inspiration for your own solution!



## Obstacle avoidance

Obstacle avoidance is an advanced topic and you will only get to this if you got trajectory tracking completely solved and working.

You have the parking sensors to help you detect obstacles, and also the perimeter of the field can be considered an (invisible) obstacle. Once you detect an obstacle, you need to steer around it. Some suggested approaches are:

- Define a new planned trajectory around the obstacle (possibly requiring driving backwards). Typically, students follow a *combinatorial approach*: lots of if-then-else statements. This works in simple cases. The disadvantage is that it is hard to debug, and the solutions are often not general.
- Study control literature on obstacle avoidance that use *artificial potential fields*. This is a general approach that essentially defines a penalty function around obstacles and then finds optimal trajectories that minimize the "cost".


